In [1]:
import pandas as pd 
import os
from os import listdir
from os.path import isfile, join, isdir
from pathlib import Path
import pickle
import xml.etree.ElementTree
import re 

WORKDIR="../"
PROYECT="full-teaching-experiment"
LINE="####################################################################"

# PROYECT="kurento"
# DIRNAME="kurento-java/kurento-integration-tests/"
# OUTPUT="../FAST/input/kurento_v0/"

In [2]:
def getCleanMessage(elem):
    return elem.get('message')#[:60].replace('\n','')

In [6]:
BUGS_PATH=WORKDIR+"Bugs/"

# TestCases
tcs=dict()

# TestCases index
n = 1

bugs = [ d for d in listdir(BUGS_PATH) if isdir(join(BUGS_PATH,d)) and d != "demo"]
bugs.sort(key=lambda v: (v[0:3], int(v[3:])))
for b in bugs:
    print(LINE+'\n'+b+'\n'+LINE)
    for ts in [ d for d in listdir(join(BUGS_PATH,b)) if isdir(join(BUGS_PATH,b,d))]:
        
        root = xml.etree.ElementTree.parse(join(BUGS_PATH,b,"%s/%s-surefire-report.xml"%(ts,ts))).getroot()
        for tc in root.iter('testcase'):
            tc_name = tc.get("name")
            
            if not tc_name in tcs:
                tc_dict = {
                    'id': n,
                    'name': tc_name,
                    'classname': tc.get("classname"),
                    'errors': [],
                    'errors_detailed': []
                }
                n=n+1
                tcs[tc_name] = tc_dict
            if tc.find('error') != None:
                tcs[tc_name]['errors'].append(int(re.search(r'\d+', b).group()))
                print("    %s::%s"%(ts, tc_name))
                print("    - ERROR: (%s) %s"%(tc.find('error').get('type'),getCleanMessage(tc.find('error'))))
                print("")
               
            
            if tc.find('failure') != None:
                tcs[tc_name]['errors'].append(int(re.search(r'\d+', b).group()))
                print("    %s::%s"%(ts, tc_name))
                print("    - FAILURE: (%s) %s"%(tc.find('failure').get('type'),getCleanMessage(tc.find('failure'))))
                print("")


####################################################################
bug1
####################################################################
    CommentControllerTest::newCommentTest
    - FAILURE: (java.lang.AssertionError) failure - expected HTTP status 401 expected:<401> but was:<201>

    CommentControllerTest::replyCommentTest
    - FAILURE: (java.lang.AssertionError) failure - expected HTTP status 401 expected:<401> but was:<201>

####################################################################
bug2
####################################################################
    FileGroupControllerTest::testModifyFileGroup
    - FAILURE: (java.lang.AssertionError) EXCEPTION: //test OK

    FullTeachingTestE2EREST::filesRestOperations
    - ERROR: (org.openqa.selenium.TimeoutException) "Dialog not closed. Reason: Edition of file group failed" (checked with condition) > Timed out after 15 seconds waiting for presence of element located by: By.xpath: //div[@id='put-delete-modal' and c

In [ ]:
# data = pd.read_csv("KurentoTest.csv")
# data = data[data['NumErrors'] == 0]
# data = data[data['NumSkipped'] == 0]
# data = data.reset_index()

In [ ]:
# def getCode(name):
#     filepath = list(Path(DIRNAME).rglob('%s.java'%name))[0]
#     with open(filepath, "r") as f:
#         content = f.read()
#     return content

In [ ]:
# # Hay que generar:
# # -> *-bbox.txt (Codigo fuente, 1 linea = 1 TestSuite)
# # -> fault_matrix.pickle
# # -> times.txt (Tiempos, 1 linea = tiempo en s de 1 TestSuite)

# tests = []
# fault_matrix = {}
# error_index = 1
# ts_index = 1

# for ts in data.iterrows():
#     name = ts[1]['TestSuite']
#     code = getCode(name)
#     test ={
#         'name': name,
#         'code': getCode(name),
#         'time': ts[1]['Time']
#     }
#     if ts[1]['NumFailure'] > 0:
#         fault_matrix[error_index] = [ts_index]
#         error_index += 1
#     tests.append(test)
#     ts_index += 1

# # BBOX WRITE
# with open(OUTPUT+PROYECT+"-bbox.txt", "w+") as out:
#     for t in tests:
#         out.write(str(t['code'].replace('\n',''))+'\n')
# # FAULT MATRIX WRITE
# with open(OUTPUT+"fault_matrix.pickle", "wb" ) as fm:
#     # Protocol 2 for Python 2.7
#     pickle.dump( fault_matrix , fm , protocol=2)
    
# # TIMES WRITE
# with open(OUTPUT+"times_avg.txt", "w" ) as tm:
#     for t in tests:
#         tm.write(str(t['time'])+'\n')